# Notebook allowing creating of VE synthetic data and conversion from coeffs to model params only

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import VE_datagen
import VE_params

np_seed = 2
torch_seed = 0
np.random.seed(np_seed)
torch.manual_seed(torch_seed)

## Data generation

> The below cell is very important for preparing the generation, examination, and saving of the data. It is one of only a few cells that requires configuration in the notebook.

Specify the model parameters

In [ ]:
input_type = 'Strain'

# For Boltzmann DG, specific model required for calculation of response given manipulation type. Strain -> GMM, Stress -> GKM.
# For odeint method, choose what you want.
mech_model = 'GMM' 
    
E = [5e-4, 5e-4, 5e-4] # All 2 kOhm resistors
eta = [220e-6, 33e-6] # 220 uF and 33 uF capacitors

In [ ]:
# # Convert to the equivalent description of model specified by 'model' which is what flow is forced to presume given input_type
# # The third arg should be the original format of the model described above, the opposite to what will be assumed by input_type
# E, eta = VE_params.convert_between_models(E_GMM, eta_GMM, 'GMM')
# print(E, eta)

In [ ]:
E_alt = np.array(E[1:])
eta_alt = np.array(eta)
tau = eta_alt/E_alt
tau

Specify the functional form of the input

In [ ]:
func_desc = 'Half Sinc'

# OLD MANIPULATION
# sinc
omega = 1
Amp = 7
input_expr = lambda t: Amp*np.sin(omega*t)/(omega*t) # For DG
d_input_expr = lambda t: (Amp/t)*(np.cos(omega*t) - np.sin(omega*t)/(omega*t)) # For DG
input_torch_lambda = lambda t: Amp*torch.sin(omega*t)/(omega*t) # For library

# YOUR MANIPULATION
#
#
#
#
#

Specify the independant data points over which to synthesise the data.

In [ ]:
time_array = np.linspace(10**-10, 10*np.pi/omega, 5000).reshape(-1, 1)

In [ ]:
strain_array, stress_array = VE_datagen.calculate_strain_stress(input_type, time_array, input_expr, E, eta, D_input_lambda=d_input_expr)

In [ ]:
# strain_array, stress_array = VE_datagen.calculate_int_diff_equation_initial(time_array, input_expr, E, eta, input_type, mech_model)

In [ ]:
plt.plot(time_array.flatten(), strain_array.flatten(), label='strain')

In [ ]:
plt.plot(time_array.flatten(), stress_array.flatten(), label='stress')

## Generation Validation

This small section uses numerical derivs to check that the data generated follows the expected diff equation. First, the true expected coeffs are calaculated.

In [ ]:
unscaled_coeffs = VE_params.coeffs_from_model_params(E, eta, mech_model)
unscaled_coeffs

Then the GDM residual is calculated at each point.

In [ ]:
errors = VE_datagen.equation_residuals(time_array, strain_array, stress_array, unscaled_coeffs)

In [ ]:
plt.semilogy(abs(errors.flatten()))

## Data Treatment

### Scaling

Presence of t/time_sf in new lambdas is hard to understand but has a reason. The reason is that while unscaled t, stress and strain all map to each other, they need to all map once scaled also. Scaling the time array does not change the target array as these are both precalculated. however, because the analytical input data is calculated based off this NEW SCALED time series in the library function, it is calculated over the scaled time series for the old function, effectively doubling the number of bumps in the curve, rather than stretching it out. we want to calculate the old input_data, ie that originates from the unscaled time data, so we have to unscale the time data on the fly in the library, hence the factor in the lambda function.

This allows PyTorch to map scaled time to scaled input and calculate the appropriate derivatives

Note, this is not an issue for the real data as there is no analytical input term, and the input variable is a dumb target, just like the output variable.

In [ ]:
# 'normalising'
time_sf = omega/1.2
strain_sf = 1/np.max(abs(strain_array))
stress_sf = 1/np.max(abs(stress_array))
print(time_sf, strain_sf, stress_sf)

scaled_time_array = time_array*time_sf
scaled_strain_array = strain_array*strain_sf
scaled_stress_array = stress_array*stress_sf
if input_type == 'Strain':
    scaled_input_expr = lambda t: strain_sf*input_expr(t/time_sf)
    scaled_input_torch_lambda = lambda t: strain_sf*input_torch_lambda(t/time_sf)
    scaled_target_array = scaled_stress_array
elif input_type == 'Stress':
    scaled_input_expr = lambda t: stress_sf*input_expr(t/time_sf)
    scaled_input_torch_lambda = lambda t: stress_sf*input_torch_lambda(t/time_sf)
    scaled_target_array = scaled_strain_array

#### Predicting Coefficients

Scale the true coeffs to what deepmod should find based on the scaling of each term in the equation.

In [ ]:
expected_coeffs = VE_params.scaled_coeffs_from_true(unscaled_coeffs, time_sf, strain_sf, stress_sf)

#### Scaling Validation

In [ ]:
errors = VE_datagen.equation_residuals(scaled_time_array, scaled_strain_array, scaled_stress_array, expected_coeffs)

In [ ]:
plt.semilogy(abs(errors.flatten()))

### Noise

In [ ]:
# add noise
noise_level = 0 # No noise ever added

noisy_target_array = scaled_target_array + noise_level * np.std(scaled_target_array) * np.random.standard_normal(scaled_target_array.shape)

### Random Sampling

In [ ]:
# sampling
number_of_samples = 1000

reordered_row_indices = np.random.permutation(scaled_time_array.size)

reduced_time_array = scaled_time_array[reordered_row_indices, :][:number_of_samples]
reduced_target_array = noisy_target_array[reordered_row_indices, :][:number_of_samples]

## Defining Library

In [ ]:
import torch.autograd as auto
    
def mech_library(inputs, **library_config):    
    
    prediction, data = inputs
    
    # Load already calculated derivatives of manipulation variable
    input_theta = library_config['input_theta']
    if data.shape[0] == 1: # Swaps real input_theta out for dummy in initialisation pass.
        input_theta = torch.ones((1, input_theta.shape[1]))
    
    # Automatic derivatives of response variable 
    output_derivs = auto_deriv(data, prediction, library_config['diff_order'])
    output_theta = torch.cat((prediction, output_derivs), dim=1)
    
    # Identify the manipulation/response as Stress/Strain and organise into returned variables
    if library_config['input_type'] == 'Strain':
        strain = input_theta
        stress = output_theta
    else: # 'Stress'
        strain = output_theta
        stress = input_theta
        
    strain_t = strain[:, 1:2] # Extract the first time derivative of strain
    strain = torch.cat((strain[:, 0:1], strain[:, 2:]), dim=1) # remove this before it gets put into theta
    strain *= -1 # The coefficient of all strain terms will always be negative. rather than hoping deepmod will find these negative terms, we assume the negative factor here and later on DeepMoD will just find positive coefficients
    theta = torch.cat((strain, stress), dim=1) # I have arbitrarily set the convention of making Strain the first columns of data
    
    return [strain_t], theta


def auto_deriv(data, prediction, max_order):
    '''
    data and prediction must be single columned tensors.
    If it is desired to calculate the derivatives of different predictions wrt different data, this function must be called multiple times.
    This function does not return a column with the zeroth derivative (the prediction).
    '''
    
    # First derivative builds off prediction.
    derivs = auto.grad(prediction, data, grad_outputs=torch.ones_like(prediction), create_graph=True)[0]
    for _ in range(max_order-1):
        # Higher derivatives chain derivatives from first derivative.
        derivs = torch.cat((derivs, auto.grad(derivs[:, -1:], data, grad_outputs=torch.ones_like(prediction), create_graph=True)[0]), dim=1)
            
    return derivs

## DeepMod prep

In [ ]:
time_tensor = torch.tensor(reduced_time_array, dtype=torch.float32, requires_grad=True)
target_tensor = torch.tensor(reduced_target_array, dtype=torch.float32)

#### Manipulation derivative library pre-calculation

In [ ]:
library_diff_order = 3

input_data = scaled_input_torch_lambda(time_tensor)
input_derivs = auto_deriv(time_tensor, input_data, library_diff_order)
input_theta = torch.cat((input_data.detach(), input_derivs.detach()), dim=1)

#### Config dictionaries

In [ ]:
library_config = {'library_func': mech_library,
                  'diff_order': library_diff_order,
                  'coeff_sign': 'positive',
                  'input_type': input_type,
                  'input_theta': input_theta}

# NOW INSERT DEEPMOD

## Rediscovering mechanical model parameters if possible

We need to take the coefficients that DeepMoD has found and reverse the process in predicting coeffients.

First we do the reverse scaling of the coeffs, this time dividing by the multiplication factor previously found, to scale the scaled coefficients to the true ones.

Below line explanation:

>final_coeffs_array needs to be the result you get.
>
>mask and library_diff_order are optional arguements and can be omitted if the set of coeffs you get back are of a viable pattern. Otherwise, to make suure correct scaling applied to correct coeff, need the mask. If in doubt, include the mask.
>
>For library_diff_order = 3, a viable second order result will have mask [0,1,3,4,5], and a 1st order result would be [0,3,4]

In [ ]:
unscaled_final_coeffs = VE_params.true_coeffs_from_scaled(final_coeffs_array, time_sf, strain_sf, stress_sf, mask=sparsity_mask_array, library_diff_order=library_diff_order)

In [ ]:
true_coeffs = list(unscaled_final_coeffs)

We next use these coefficients to recover our model parameters.

Note, if sparsity mask describes an inviable model, this may still run, but results are nonsense as assumption is made about which values correspond to which coeffs, which is not followed for an invalid mask.

In [ ]:
recovered_mech_params = VE_params.model_params_from_coeffs(true_coeffs, mech_model, True)
recovered_mech_params